# Assignment 2

This assignment serves as a comprehensive evaluation of your machine learning skills, encompassing not only the technical aspects of model development but also your ability to analyze, interpret, and present data insights effectively. As such, it's essential to ensure that your submission is complete, functional, and devoid of any obvious gaps, as if you were delivering this project to a client.

To achieve this, leverage the full capabilities of Markdown and the interactive visualization tools available in Jupyter notebooks to craft a well-structured and visually appealing report of your findings. Your report should clearly communicate the insights you've gained from the exploratory data analysis, the rationale behind your data preprocessing and feature engineering decisions, and a thorough analysis of feature importance. High-quality visualizations and well-organized documentation will not only support your analysis but also make your results more accessible and understandable to your audience.

Remember, the ability to present complex results in an intuitive and engaging manner is a crucial skill, almost as important as the technical proficiency in model building and data analysis. Treat this assignment as an opportunity to showcase your skills in both areas.

## Instructions
- Your submission should be a `.ipynb` file with your name,
  like `FirstnameLastname.ipynb`. It should include the answers to the questions in markdown cells, your data analysis and results.
- You are expected to follow the best practices for code writing and model
training. Poor coding style will be penalized.
- You are allowed to discuss ideas with your peers, but no sharing of code.
Plagiarism in the code will result in failing. If you use code from the
internet, cite it by adding the source of the code as a comment in the first line of the code cell. [Academic misconduct policy](https://wiki.innopolis.university/display/DOE/Academic+misconduct+policy)
- In real life clients can give unclear goals or requirements. So, if the instructions seem vague, use common sense to make reasonable assumptions and decisions.

## Self-Reliance and Exploration
In this task, you're encouraged to rely on your resourcefulness and creativity. Dive into available resources, experiment with various solutions, and learn from every outcome. While our team is here to clarify task details and offer conceptual guidance, we encourage you to first seek answers independently. This approach is vital for developing your problem-solving skills in machine learning.



# Task 2: Image Classification with CNNs (50%)

In this task, you'll dive into the world of Convolutional Neural Networks (CNNs) by working with the CIFAR-10 dataset, a staple in image classification challenges. Your goal is to build and evaluate two different CNN models to classify images into one of the ten categories accurately.

The dataset is availabel in pytorch and keras.

## Part 1: Custom CNN Model (20%)

- Design and train a CNN model from scratch tailored for the CIFAR-10 dataset.
- Focus on the architecture that you believe will perform best for this specific task.
- Integrate various techniques such as batch normalization, dropout, learning rate schedulers, and early stopping to improve model training. Experiment with these methods and finetune them to see how they affect training stability, convergence speed, and overall performance.

## Part 2: Transfer Learning Model (20%)

- Implement a transfer learning approach using a pre-trained model of your choice.
- Fine-tune the model on the CIFAR-10 dataset to achieve the best possible performance.

## Evaluation (10%)

Ensure that both models are robust and generalized well to unseen data.

After training both models, you will evaluate them on a provided test dataset.

Compare your models based on:
- **AUC-ROC**: How well does each model discriminate between classes?
- **Model Size**: Consider the trade-offs in model complexity.
- **Inference Speed**: Evaluate how quickly your model can predict classes for new images.

Reflect on the performance, size, and inference speed of both models. What insights can you draw from these comparisons?

### Learning Objectives

- Understand and apply CNNs for image classification.
- Explore the impact of model architecture on performance and efficiency.
- Learn the process and benefits of transfer learning in deep learning.

Remember, the key to this task is not just about achieving the highest accuracy but also understanding the strengths and limitations of different approaches in machine learning model development.

In [1]:
# import libraries
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import models
from torch.optim import SGD
from torch.optim import lr_scheduler
from time import time
from tqdm import tqdm
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Normalize
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from sklearn.metrics import roc_auc_score
import time

The train and test function were inspired by code from lab-10

In [2]:
def train(model, device, train_loader, criterion, optimizer, epoch):
    """
    Train the model for one epoch
    :param model: model to be trained
    :param device: device to be used for training
    :param train_loader: train data loader
    :param criterion: criterion to be used for training
    :param optimizer: optimizer to be used for training
    :param epoch: epoch number
    :return: (train_loss, train_acc)
    """
    model.train()
    epoch_loss = 0
    start_time = time.time()
    correct = 0
    iteration = 0

    bar = tqdm(train_loader)
    for data, target in bar:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()

        output = model(data)
        # Get the index of the max log-probability
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        iteration += 1
        bar.set_postfix({"Loss": format(epoch_loss/iteration, '.6f')})

    acc = 100. * correct / len(train_loader.dataset)
    print(f'\rTrain Epoch: {epoch}, elapsed time:{time.time()-start_time:.2f}s')
    return epoch_loss, acc


def test(model, device, test_loader, criterion):
    """
    Test the model for one epoch
    :param model: model to be tested
    :param device: device to be used for testing
    :param test_loader: test data loader
    :param criterion: criterion to be used for testing
    :return: (test_loss, test_acc)
    """
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    acc = 100. * correct / len(test_loader.dataset)
    return test_loss, acc

In [3]:
class CNN_from_scratch(nn.Module):
    """
    CNN from scratch
    """
    def __init__(self):
        super(CNN_from_scratch, self).__init__()
        self.conv0 = nn.Conv2d(3, 32, (3,3), padding="same")
        self.batchnorm0 = nn.BatchNorm2d(32)
        self.activation0 = nn.ReLU()
        self.pool0 = nn.MaxPool2d((2, 2))
        self.dropout0 = nn.Dropout(0.25)

        self.conv1 = nn.Conv2d(32, 64, (3,3), padding="same")
        self.batchnorm1 = nn.BatchNorm2d(64)
        self.activation1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d((2, 2))
        self.dropout1 = nn.Dropout(0.25)

        self.conv2 = nn.Conv2d(64, 64, (3,3), padding="same")
        self.batchnorm2 = nn.BatchNorm2d(64)
        self.activation2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d((2, 2))
        self.dropout2 = nn.Dropout(0.25)

        self.flat = nn.Flatten()
        self.linear0 = nn.Linear(64*16, 512)
        self.batchnorm3 = nn.BatchNorm1d(512)
        self.activation3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.5)
        self.output = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv0(x)
        x = self.batchnorm0(x)
        x = self.activation0(x)
        x = self.pool0(x)
        x = self.dropout0(x)

        x = self.conv1(x)
        x = self.batchnorm1(x)
        x = self.activation1(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.batchnorm2(x)
        x = self.activation2(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        x = self.flat(x)
        x = self.linear0(x)
        x = self.batchnorm3(x)
        x = self.activation3(x)
        x = self.dropout3(x)

        x = self.output(x)
        return x


In [4]:
# using gpu with cuda if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [5]:
# shifting model to run on selected device
model = CNN_from_scratch().to(device)

I used transformations for the image from https://sidthoviti.com/fine-tuning-resnet50-pretrained-on-imagenet-for-cifar-10/

In [6]:
# transformations on an images from train and test sets
train_batch_size = 32
test_batch_size = 32

# Put augmentations
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=2),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
# Do not modify test transforms, because it will corrupt test data
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [7]:
# loading CIFAR-10 dataset
train_dataset = CIFAR10(root='cifar10', train=True, transform=train_transforms,download=True)
test_dataset = CIFAR10(root='cifar10', train=False, transform=test_transforms, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=train_batch_size,
                          num_workers=2, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=test_batch_size,
                         num_workers=2, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [9]:
def training(selected_model, selected_model_name, tr_loader, tst_loader, chosen_criterion, chosen_optimizer, chosen_scheduler, epochs, early_stopping_patience):
    """
    Start the training process
    :param selected_model: selected model to be trained and evaluated
    :param selected_model_name: model name to save
    :param tr_loader: train data loader
    :param tst_loader: test data loader
    :param chosen_criterion: chosen criterion to be used for training
    :param chosen_optimizer: chosen optimizer to be used for training
    :param chosen_scheduler: chosen scheduler to be used for training
    :param epochs: epochs to be used for training
    :param early_stopping_patience:
    :return: none
    """
    count = 0
    min_test_loss = np.inf
    for epoch in range(0, epochs):
        train_loss, train_acc = train(selected_model, device, tr_loader, chosen_criterion, chosen_optimizer, epoch)
        # Update learning rate if needed
        chosen_scheduler.step(train_loss)
        test_loss, test_acc = test(selected_model, device, tst_loader, chosen_criterion)
        if test_loss < min_test_loss:
          min_test_loss = test_loss
          count = 0
          torch.save(selected_model.state_dict(), f"{selected_model_name}.pt")
        else:
          count += 1
          if early_stopping_patience < count:
            print("Early stopping, restoring the model")
            selected_model.load_state_dict(torch.load(f"{selected_model_name}.pt"))
            break
        print(f"Training accuracy {train_acc}, test accuracy {test_acc}")
        print(f"Training loss {train_loss}, test loss {test_loss}")
    torch.save(selected_model.state_dict(), f"{selected_model_name}.pt")



In [ ]:
# starting training of a custom CNN
criterion = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.3, patience=2, min_lr=0.0001)
training(model, "from_scratch", train_loader, test_loader, criterion, optimizer, scheduler, 100, 10)

100%|██████████| 1563/1563 [00:14<00:00, 105.06it/s, Loss=1.717430]

Train Epoch: 0, elapsed time:14.88s


Training accuracy 39.982, test accuracy 53.7
Training loss 2684.3427271842957, test loss 393.1314752101898


100%|██████████| 1563/1563 [00:14<00:00, 106.46it/s, Loss=1.333396]

Train Epoch: 1, elapsed time:14.69s


...

Training accuracy 81.252, test accuracy 84.07
Training loss 844.7361828237772, test loss 144.4091468155384


100%|██████████| 1563/1563 [00:14<00:00, 107.17it/s, Loss=0.541067]

Train Epoch: 86, elapsed time:14.59s


Training accuracy 80.998, test accuracy 83.82
Training loss 845.6875160336494, test loss 147.84671030938625


 53%|█████▎    | 832/1563 [00:07<00:06, 110.29it/s, Loss=0.543111]

In [ ]:
# transfer learning using ResNet50 adjusting the pretrained model to suit my goals
num_classes = 10
model_pretrained = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model_pretrained.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=True)
num_features = model_pretrained.fc.in_features
model_pretrained.fc = nn.Linear(num_features, num_classes)
model_pretrained = model_pretrained.to(device)

In [ ]:
# Define loss function and optimizer
criterion_pretrained = nn.CrossEntropyLoss()
optimizer_pretrained = SGD(model_pretrained.parameters(), lr=0.001, momentum=0.9)
scheduler_pretrained = lr_scheduler.ReduceLROnPlateau(optimizer_pretrained, 'min', factor=0.5, patience=2, min_lr=0.0001)

In [ ]:
training(model_pretrained, "model_pretrained", train_loader, test_loader, criterion_pretrained, optimizer_pretrained, scheduler_pretrained, 20, 5)

100%|██████████| 1563/1563 [00:51<00:00, 30.46it/s, Loss=0.380618]

Train Epoch: 2, elapsed time:51.32s


Training accuracy 86.788, test accuracy 86.12
Training loss 594.9052270166576, test loss 131.14507415145636


100%|██████████| 1563/1563 [00:51<00:00, 30.59it/s, Loss=0.291909]

Train Epoch: 3, elapsed time:51.10s


...

100%|██████████| 1563/1563 [00:51<00:00, 30.47it/s, Loss=0.081582]


Train Epoch: 10, elapsed time:51.29s
Training accuracy 97.236, test accuracy 89.95
Training loss 127.51285666972399, test loss 115.3394356938079


100%|██████████| 1563/1563 [00:51<00:00, 30.48it/s, Loss=0.066271]

Train Epoch: 11, elapsed time:51.28s


Early stopping, restoring the model


### Validation set

In [26]:
# loading validation dataset
task_2_images = np.load("task_2_test_images.npy")
task_2_images = np.transpose(task_2_images, (0, 3, 1, 2))
task_2_images = task_2_images/255. 
task2_labels = np.load("task_2_test_labels.npy")

In [27]:
# Transform the images
normalization_transform = transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
task_2_images = normalization_transform(torch.tensor(task_2_images, dtype=torch.float32))
task2_labels = torch.tensor(task2_labels.astype("int64"))


In [28]:
def print_AUC_ROC_score(selected_model, images, labels):
    """
    Print ROC AUC score
    :param selected_model: selected model to print metrics
    :param images: images loaded
    :param labels: labels loaded
    :return: nothing
    """
    selected_model.eval()
    with torch.no_grad():
        time_start = time.time()
        y_pred = selected_model(images.to(device))
        time_spent = time.time()-time_start
        auc_roc_score = roc_auc_score(labels.cpu().numpy(), F.softmax(y_pred, dim=1).cpu().numpy(), multi_class='ovr')
        print("AUC-ROC Score:", auc_roc_score)
        print("Time spent in seconds: ", time_spent)

In [29]:
print_AUC_ROC_score(model, task_2_images, task2_labels)

AUC-ROC Score: 0.9496791442659868
Time spent in seconds:  0.017938852310180664


In [30]:
print_AUC_ROC_score(model_pretrained, task_2_images, task2_labels)

AUC-ROC Score: 0.9729164026933512
Time spent in seconds:  0.02630472183227539


Performance: The pretrained ResNet50 model outperforms the model trained from scratch, as evidenced by the higher AUC-ROC score.

Model Size: While the pretrained ResNet50 model have a larger model size due to its architecture and pretrained weights, it offers superior performance.

Inference Speed: The model trained from scratch shows faster inference speed, which could be advantageous in scenarios where real-time predictions are critical.

These comparisons indicate that although the model trained from scratch may offer advantages in terms of being lightweight and faster during inference, the pretrained ResNet50 model demonstrates superior classification performance. The selection between these two models hinges on the specific needs of the application, weighing factors such as performance, model size, and inference speed.